In [1]:
# set theme
!jt -t oceans16

In [2]:
# this is to make the Jupyter notebook as wide as the screen (on the Mac at least)
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%config InlineBackend.figure_format ='retina'

### Import needed modules

In [3]:
import numpy as np
import xarray as xr
import scipy.interpolate as spint
import os
import glob

# Plotting
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib as mpl

import ipdb

# from pylab import *
from matplotlib import dates

from datetime import datetime, timedelta
from datetime import date

import cmocean
import matplotlib.dates as mdates

# import holteandtalley as ht
import gsw

from bgc import plot_bbp, plot_flags, medfilt1

# from BGC_Argo_BBP_RTQC import BBP_Spike_test, global_range_test_BBP700
# from BGC_Argo_BBP_RTQC import global_range_test_BBP700 #, BBP_Spike_test

import subprocess

import warnings
warnings.filterwarnings('ignore')
    
# import inspect #https://www.stefaanlippens.net/python_inspect/
import pickle

import gc
from parfor import pmap


In [4]:
# global variables defined inside tests and used elsewhere
global A_MIN_BBP700, A_MAX_BBP700

In [5]:
# function to define adaptive median filtering based on Christina Schallemberg's suggestion for CHLA
def adaptive_medfilt1(x, y, PLOT=False):
    # applies a median filtering followin Christina Schlallemberb's recommendations
    # x is PRES
    # y is BBP
    
    
#     x = PRES[innan]
#     y = BBP700[innan]

    # compute x resolution
    xres = np.diff(x)

    # initialise medfiltered array
    ymf = np.zeros(y.shape)*np.nan

    ir_LT1 = np.where(xres<1)[0]
    if np.any(ir_LT1):
        win_LT1 = 11.
        ymf[ir_LT1] = medfilt1(y[ir_LT1], win_LT1)   

    ir_13 = np.where((xres>=1) & (xres<=3))[0]
    if np.any(ir_13):
        win_13 = 7.
        ymf[ir_13] = medfilt1(y[ir_13], win_13)   

    ir_GT3 = np.where(xres>3)[0]
    if np.any(ir_GT3):
        win_GT3 = 5.
        ymf[ir_GT3] = medfilt1(y[ir_GT3], win_GT3)   

    if PLOT:
        plt.plot(np.log10(y), x, 'o')
        plt.plot(np.log10(ymf), x, 'r-')
    
    return ymf


# Implement different RTQC for BBP data using B-files

## Tests that do not remove the entire profile

### GLOBAL RANGE TEST (test order code "A")
<br>
#### Objective:
To detect and flag values of BBP532 and BBP700 that are outside the expected range.
<br>
<br>
#### What is done:
Check that:<br>
<code> medfilt1(BBP700) </code>  is in range [0, 0.001] m$^{-1} $.<br>
<code> medfilt1(BBP532) </code> is in range [0, 0.001] m$^{-1} $.<br>
<br>
The value of <code>A_MAX_BBP700=0.001</code> is taken as a very conservative estimate based on histograms in fig 2 of Bisson et al., 2019, 10.1364/OE.27.030191
<br>
<br>
#### QC flag if test fails
3
<br>
Note: the entire profile is flagged, if any negative point is found (because if there is a negative value, then
it is worth looking at the profile during DMQC).
<br>
__________________________________________________________________________________________


In [41]:
##################################################################
##################################################################
# test parameters defined outside function to make them global variables
A_MIN_BBP700 = 0 # [1/m]
A_MAX_BBP700 = 0.01 # [1/m] REVISED VALUE (very conservative estimate based on histograms in fig 2 of Bisson et al., 2019, 10.1364/OE.27.030191)
#     MAX_BBP700 = 0.1 # [1/m] NOTE that at least for BBP700 in 6901152, this value is above the saturation of the instrument (~0.077 1/m)

def BBP700_Global_range_test(BBP, BBPmf1, PRES, QC_Flags, QC_1st_failed_test, fn, VERBOSE=False, PLOT=False):
    # BBP: nparray with all BBP data
    # BBPmf1: median-filtered BBP data
    # QC_Flags: array with QC flags
    # QC_flag_1st_failed_test: array with info on which test failed QC_TEST_CODE
    # fn: name of corresponding B-file
    #
    # WHAT IS DONE: Then this tests fails, only the failing points are flagged (QC=3)
   
    FORCE_PLOT = False # this is to plot even if the test does not fails
    
    QC = 3
    QC_TEST_CODE = 'A'    
#     ISBAD = np.array([])
    ISBAD = np.zeros(len(BBPmf1), dtype=bool) # flag for noisy profile

     
    # this is the test 
    ibad = np.where( (BBPmf1 > A_MAX_BBP700) | (BBPmf1 < A_MIN_BBP700) | (BBP < A_MIN_BBP700) )[0]
    ISBAD[ibad] = 1
    if np.any(ISBAD==1): # If ISBAD is not empty
        # flag entire profile if any negative value is found
        if np.any(BBPmf1 < A_MIN_BBP700):
            ISBAD = np.where(BBPmf1)

        # apply flag
        QC_Flags[ISBAD] = QC
        QC_1st_failed_test[ISBAD] = QC_TEST_CODE
        
        if VERBOSE:
            print('Failed Global_Range_test')
            print('applying QC=' + str(QC) + '...')
            
        if PLOT:
            plot_failed_QC_test(BBP, BBPmf1, PRES, ISBAD, QC_Flags, QC_1st_failed_test, QC_TEST_CODE, fn, VERBOSE, FORCE_PLOT)
        
    return QC_Flags, QC_1st_failed_test


### SPIKY PROFILE TEST (test order code "D")
<br>
#### Objective:
To detect and flag spikes in BBP700 that are outside the expected range.
<br>
<br>
#### What is done:
Compute  <code> absres = abs(BBP-medfilt1(BBP, 31))</code>. <br>
Check that:
<code> absres < 0.001 </code> m$^{-1}$. <br>
<br>
<br>
#### QC flag if test fails
2
    <br>
__________________________________________________________________________________________

In [7]:
#################################################################
#################################################################
def BBP_Spiky_Profile_test(BBP, BBPmf1, PRES, QC_Flags, QC_1st_failed_test, fn, VERBOSE=False, PLOT=True):
    # BBP: nparray with all BBP data
    # BBPmf1: smooth BBP array (medfilt1(BBP700, 31)
    # QC_Flags: array with QC flags
    # QC_flag_1st_failed_test: array with info on which test failed QC_TEST_CODE
    # fn: name of corresponding B-file
    #
    # WHAT IS DONE: When the test fails, only the failed points are flagged (QC=3)
    
    QC = 2; # flag to apply if the result of the test is true
    QC_TEST_CODE = 'D'
    ISBAD = np.array([]) # flag for noisy data
#     REL_RES_THRESHOLD = 0.5 # threshold for relative residuals
    RES_THRESHOLD = 0.001 # [1/m] threshold for absolute residuals
    
    FORCE_PLOT = False # plot even if the test does not fail (used when developing the test)

#     rel_res = np.empty(BBP.shape)
#     rel_res[:] = np.nan
    
    absres = np.empty(BBP.shape)
    absres[:] = np.nan
    


    innan = np.where(~np.isnan(BBP))[0]
#     ipdb.set_trace()
    
    if len(innan)>10: # if we have at least 10 points in the profile

#         rel_res[innan] = BBP[innan]/BBPmf1[innan]-1
        absres[innan] = np.abs(BBP[innan]-BBPmf1[innan])

#         ioutliers = np.where(abs(rel_res)>REL_RES_THRESHOLD)[0] # index of where the rel res are greater than the threshold
        ioutliers = np.where(absres > RES_THRESHOLD)[0] # index of where the rel res are greater than the threshold

        ISBAD = ioutliers

    # update QC_Flags to 3 when bad profiles are found
    if len(ISBAD)>0:     
        iQChigher = np.where(QC_Flags < QC) # but first check that there are no QCflags > than the one we want to assign in this profile
#         iQChigher = np.where(QC_Flags < 100) # but first check that there are no QCflags > than the one we want to assign in this profile
        if iQChigher:
            if VERBOSE:
                print('Failed BBP_Spiky_Profile_test')
                print('applying QC=' + str(QC) + '...')
                
            QC_Flags[iQChigher] = QC
            QC_1st_failed_test[iQChigher] = QC_TEST_CODE

    if ((len(ISBAD)>0) & (PLOT==True)) | (FORCE_PLOT==True):
#                 plot_failed_QC_test(rel_res, PRES, ISBAD, QC_Flags, QC_1st_failed_test, QC_TEST_CODE, fn)
        plot_failed_QC_test(absres, absres*0., PRES, ISBAD, QC_Flags, QC_1st_failed_test, QC_TEST_CODE, fn, VERBOSE, FORCE_PLOT)

    
#     return QC_Flags, QC_1st_failed_test, rel_res
    return QC_Flags, QC_1st_failed_test, absres


## Tests that flag the entire profile

### BBP NOISY-PROFILE TEST  (test order code "B")
<br>
#### Objective:
To detect and flag profiles of BBP that are affected by noisy data.
<br><br>
#### What is done:
Compute <code> res = abs(BBP-medfilt1(BBP, 31))</code>.

Flag profiles where at least <code>15%</code> of the profile has <code>res > 0.001</code>  m$^{-1}$.<br>
<br><br>
#### QC flag if test fails
3
<br>
__________________________________________________________________________________________

In [8]:
##################################################################
##################################################################
B_RES_THRESHOLD = 0.001 # threshold for relative residuals
B_FRACTION_OF_PROFILE_THAT_IS_OUTLIER = 0.15 # fraction of profile with relative residuals above REL_RES_THRESHOLD

def BBP_Noisy_Profile_test(BBP, BBPmf1, PRES, QC_Flags, QC_1st_failed_test, fn, VERBOSE=False, PLOT=True):
    # BBP: nparray with all BBP data
    # BBPmf1: smooth BBP array (medfilt1(BBP700, 31)
    # QC_Flags: array with QC flags
    # QC_flag_1st_failed_test: array with info on which test failed QC_TEST_CODE
    # fn: name of corresponding B-file
    #
    # WHAT IS DONE: When the test fails, all points in the profile are flagged (QC=3)
    
    QC = 3; # flag to apply if the result of the test is true
    QC_TEST_CODE = 'B'
    ISBAD = np.array([]) # flag for noisy profile
#     REL_RES_THRESHOLD = 0.5 # threshold for relative residuals
    
    FORCE_PLOT = False # plot even if the test does not fail (used when developing the test)

#     rel_res = np.empty(BBP.shape)
#     rel_res[:] = np.nan
    
    res = np.empty(BBP.shape)
    res[:] = np.nan
    


    innan = np.where(~np.isnan(BBP))[0]
#     ipdb.set_trace()
    
    if len(innan)>10: # if we have at least 10 points in the profile

#         rel_res[innan] = BBP[innan]/BBPmf1[innan]-1
        res[innan] = np.abs(BBP[innan]-BBPmf1[innan])

#         ioutliers = np.where(abs(rel_res)>REL_RES_THRESHOLD)[0] # index of where the rel res are greater than the threshold
        ioutliers = np.where(abs(res)>B_RES_THRESHOLD)[0] # index of where the rel res are greater than the threshold

        if len(ioutliers)/len(innan)>=B_FRACTION_OF_PROFILE_THAT_IS_OUTLIER: # this is the actual test: are there more than a certain fraction of points that are noisy?
            ISBAD = ioutliers

    # update QC_Flags to 3 when bad profiles are found
    if len(ISBAD)>0:     
        iQChigher = np.where(QC_Flags < QC) # but first check that there are no QCflags > than the one we want to assign in this profile
#         iQChigher = np.where(QC_Flags < 100) # but first check that there are no QCflags > than the one we want to assign in this profile
        if iQChigher:
            if VERBOSE:
                print('Failed BBP_Noisy_Profile_test')
                print('applying QC=' + str(QC) + '...')
                
            QC_Flags[iQChigher] = QC
            QC_1st_failed_test[iQChigher] = QC_TEST_CODE

    if ((len(ISBAD)>0) & (PLOT==True)) | (FORCE_PLOT==True):
#                 plot_failed_QC_test(rel_res, PRES, ISBAD, QC_Flags, QC_1st_failed_test, QC_TEST_CODE, fn)
        plot_failed_QC_test(res, res*0., PRES, ISBAD, QC_Flags, QC_1st_failed_test, QC_TEST_CODE, fn, VERBOSE, FORCE_PLOT)

    
#     return QC_Flags, QC_1st_failed_test, rel_res
    return QC_Flags, QC_1st_failed_test, res


### BBP HIGH-DEEP-VALUES TEST  (test order code "C")
<br>
#### Objective:
To detect and flag profiles of BBP that have at least some (5) points anomalously high values at depth. It could indicate multiple problems: wrong calibration coefficients, biofouling, bad sensor, grounding, etc.
<br><br>
#### What is done:
Check if <code>median(BBP700)</code> below <code>800 dbars</code> is above a threshold of <code>0.0005 </code> m$^{-1}$.  (this is half the value typical for surface bbp in the oligotrophic ocean, smoothed bbp data at depth are expected to be lower than this value).

Flag entire profile.
<br><br>
#### QC flag if test fails
3
<br>
__________________________________________________________________________________________

In [9]:
##################################################################
##################################################################
C_DEPTH_THRESH = 800 #[dbars] below this threshold we consider it "deep"
C_DEEP_BBP700_THRESH = 0.0005 # [1/m] threshold for bbp at depth
C_N_of_ANOM_POINTS = 5 # number of anomalous points required for the test to fail

def BBP700_High_Deep_Values_test(BBPmf1, PRES, QC_Flags, QC_1st_failed_test, fn, VERBOSE=False, PLOT=True):
    # BBP: nparray with all BBP data
    # BBPmf1: smooth BBP array (medfilt1(BBP700, 31)
    # QC_Flags: array with QC flags
    # QC_flag_1st_failed_test: array with info on which test failed QC_TEST_CODE
    # fn: name of corresponding B-file
    #
    # WHAT IS DONE: When the test fails, all points in the profile are flagged (QC=3)
    
    FORCE_PLOT = False # this is to plot even if the test does not fails
        
    QC = 3; # flag to apply if the result of the test is true
    QC_TEST_CODE = 'C'
    ISBAD = np.zeros(len(BBPmf1), dtype=bool) # flag for noisy profile
    
    # this is the test 
    iDEEP = np.where(PRES>C_DEPTH_THRESH)
    if (np.nanmedian(BBPmf1[iDEEP]) > C_DEEP_BBP700_THRESH) & ( len(BBPmf1[iDEEP]) >= C_N_of_ANOM_POINTS):
        ISBAD = np.ones(len(BBPmf1), dtype=bool)
    
    if np.any(ISBAD==1): # if ISBAD, then apply QC_flag=3  
        iQChigher = np.where(QC_Flags < QC) # but first check that there are no QCflags > than the one we want to assign in this profile
        if iQChigher:
            if VERBOSE:
                print('Failed High_Deep_Values_test')
                print('applying QC=' + str(QC) + '...')

                
            QC_Flags[iQChigher] = QC
            QC_1st_failed_test[iQChigher] = QC_TEST_CODE

            if PLOT:
                plot_failed_QC_test(BBPmf1, BBPmf1, PRES, ISBAD, QC_Flags, QC_1st_failed_test, QC_TEST_CODE, fn, VERBOSE, FORCE_PLOT)



    return QC_Flags, QC_1st_failed_test


### BBP MISSING-DATA TEST  (test order code "E")
<br>
#### Objective:
To detect and flag profiles of BBP that have a large fraction of missing data. This test can also detect profiles that are too shallow with respect to the Argo mission.

<br><br>
#### What is done:
Ensure that we have at least 1 measurements every 100 dbars in the upper 1000 dbars.

Flag entire profile.
<br><br>
#### QC flag if test fails
2 if continuous bins from the surface (shallow profile)<br>
3 if less than 2 data points in 100-dbar bins from 0 to 1000 dbars (because there maybe good profiles that are shallower than 1000 dbars)<br>
4 if only data within one bin
<br>
__________________________________________________________________________________________

In [10]:
##################################################################
##################################################################
def BBP700_Missing_Data_test(BBP, PRES, QC_Flags, QC_1st_failed_test, fn, VERBOSE=False, PLOT=True):
    # BBP: nparray with all BBP data
    # QC_Flags: array with QC flags
    # QC_flag_1st_failed_test: array with info on which test failed QC_TEST_CODE
    # fn: name of corresponding B-file
    #
    # WHAT IS DONE: When the test fails, all points in the profile are flagged (QC=2)
    
    FORCE_PLOT = False # this is to plot even if the test does not fails
        
    QC_all = [np.nan, np.nan, np.nan]    
    QC_all[0] = 2 # flag to apply if we have less than 10 consecutive bins from the surface
    QC_all[1] = 3 # flag to apply if the result of the test is true
    QC_all[2] = 4 # flag to apply if there are data only within one size bin
    QC_TEST_CODE = 'E'
    MIN_N_PERBIN = 1 # minimum number of points in each bin
    ISBAD = 0 # flag for noisy profile
    
    # this is the test (check also that the corresponding flag have not been changed from the initial QC=1)
    # bin the profile into 100-dbars bins
    bins = np.linspace(50, 1000, 10) # create 10 bins between 0 and 1000 dbars
    PRESbin = np.digitize(PRES, bins) # assign PRES values to each bin
    bin_counts = np.zeros(bins.shape)*np.nan
    for i in range(len(bins)):
        if i==0:
            bin_counts[i] = len(np.where(PRES<bins[i])[0])
        else:
            bin_counts[i] = len(np.where((PRES>=bins[i-1]) & (PRES<bins[i]))[0])
            
#     bin_counts = np.asarray([np.count_nonzero(BBP[PRESbin == i]>0) for i in range(len(bins))]) # count how many data points per bin there are

    # this is the actual test
    if np.any(np.nonzero(bin_counts<MIN_N_PERBIN)[0]):
        ISBAD = 1  
        
        # these statement below are for QC=2
        nonempty = np.where(bin_counts>0)[0] # index of bins with data inside
        if len(nonempty)>1:
            test_bin = np.linspace(0, nonempty[-1], nonempty[-1]+1) # create array with consecutive indices from 0 to the last element of nonempty
            # if there is only one bin with data then QC=4
            if len(np.nonzero(bin_counts>MIN_N_PERBIN)[0])==1: 
                print("data only in one bin: QC=4")
                QC = QC_all[2]

            # if there are consecutive bins from zero index (i.e. shallow profile) 
            # and if not all bins contain data, then QC=2    
            elif (np.all(test_bin==nonempty)) & (nonempty[-1]<9):  # nonempty[0]>0)
                print("shallow profile: QC=2")
                QC = QC_all[0]

            # if missing data in the profile, but not cosecutively from bottom, QC=3        
            else:
                print("data in some bins missing: QC=3")
                QC = QC_all[1]
                
        else: # this is for when we have no data at all, QC=4
            print("no data at all: QC=4")
            QC = QC_all[2]


                
    if ISBAD==1: # if ISBAD, then apply QC_flag  
        iQChigher = np.where(QC_Flags < QC) # but first check that there are no QCflags > than the one we want to assign in this profile
        if iQChigher:
            if VERBOSE:
                print('Failed Missing_Data_test')
                print('applying QC=' + str(QC) + '...')

                
            QC_Flags[iQChigher] = QC
            QC_1st_failed_test[iQChigher] = QC_TEST_CODE

            if PLOT:
                plot_failed_QC_test(BBP, bin_counts, PRES, ISBAD, QC_Flags, QC_1st_failed_test, QC_TEST_CODE, fn, VERBOSE, FORCE_PLOT)



    return QC_Flags, QC_1st_failed_test


## Apply tests

### Create lists of WMOs that include BBP meters on different float types

In [11]:
MAIN_DIR = '/data/datasets/Argo/Original/'
WORKING_DIR = '/fast_scratch/BGCArgo_plot/source/'

base_cmdo = "grep BBP700 " + MAIN_DIR + "dac/argo_bio-profile_index.txt | cut -d '/'"
# cmdo = base_cmdo + " -f2 | uniq"

# WMO = subprocess.run(cmdo, shell=True, stdout=subprocess.PIPE).stdout.decode('utf-8').split('\n')


# cmdo2 = base_cmdo + " -f1,2 | uniq"
# #aoml soccom floats
# cmdo2 = "grep BBP700 /data/datasets/Argo/Original/dac/argo_bio-profile_index.txt | awk -F \",\" '$3<=-35' |   cut -d '/' -f1,2  | uniq | grep aoml" # these are SOCCOM floats
# #aoml non-soccom floats
# cmdo2 = "grep BBP700 /data/datasets/Argo/Original/dac/argo_bio-profile_index.txt | awk -F \",\" '$3>-35' |   cut -d '/' -f1,2  | uniq | grep aoml" # these are SOCCOM floats
# #non-aoml floats
# # cmdo2 = "grep BBP700 /data/datasets/Argo/Original/dac/argo_bio-profile_index.txt |   cut -d '/' -f1,2  | uniq | grep  aoml" # these are SOCCOM floats
cmdo2 = "grep BBP700 /data/datasets/Argo/Original/dac/argo_bio-profile_index.txt |   cut -d '/' -f1,2  | uniq "  # all data
wmo_dac = subprocess.run(cmdo2, shell=True, stdout=subprocess.PIPE).stdout.decode('utf-8').split('\n')

WMO_dac = wmo_dac[::20]

# add my floats
# WMO_dac.append('bodc/6901174')
# WMO_dac.append('bodc/6901152') #metbio002
# WMO_dac.append('bodc/3901498')


WMO_dac = np.asarray(WMO_dac[::])
print(len(WMO_dac))
# type(WMO_dac)
# istart = int(np.where(WMO_dac=="csiro/5905397")[0])
istart = 0
print(istart)

31
0


In [12]:
WMO_dac

array(['aoml/1902303', 'aoml/5904181', 'aoml/5904478', 'aoml/5904687',
       'aoml/5904857', 'aoml/5905080', 'aoml/5905366', 'aoml/5905969',
       'aoml/5906002', 'aoml/5906206', 'aoml/5906246', 'aoml/5906313',
       'bodc/3901498', 'coriolis/4901805', 'coriolis/6901482',
       'coriolis/6901517', 'coriolis/6901580', 'coriolis/6901659',
       'coriolis/6901862', 'coriolis/6902666', 'coriolis/6902827',
       'coriolis/6902954', 'coriolis/6903273', 'coriolis/6903711',
       'coriolis/7900591', 'csiro/5903630', 'csiro/5905397',
       'incois/2902114', 'incois/2902195', 'incois/2902245',
       'jma/2903329'], dtype='<U16')

In [42]:
##################################################################
##################################################################
# function to plot results of applying test to dataset
def plot_failed_QC_test(BBP, BBPmf1, PRES, ISBAD, QC_Flags, QC_1st_failed_test, QC_TEST_CODE, fn, VERBOSE=False, FORCE_PLOT=False):
    # BBP: nparray with all BBP data
    # BBPmf1: median-filtered BBP data
    # ISBAD: index marking the data that failed the QC test
    # QC_Flags: array with QC flags
    # QC_flag_1st_failed_test: array with info on which test failed QC_TEST_CODE
    # QC_TEST_CODE: code of the failed test
    #FORCE_PLOT: flag to plot the test plot even if the test has not failed
    
    DIR_OUT = './plots/'

#     %matplotlib inline   
#     plt.ioff() # this is needed to avoid showing the plot (i.e. only saving it)
    
    fig = plt.figure(figsize=(10, 10))

    ax1 = fig.add_subplot(1,1,1)

    innan = np.nonzero(~np.isnan(BBP))
    
    
    # check that there are enough data to plot
    if len(BBP)<2:
        if VERBOSE:
            print("not enough data to plot... exiting")
        return
    

    if (not (QC_TEST_CODE == "0")) & (FORCE_PLOT == False): # this is to plot the original profile 
        ax1.plot(BBP[ISBAD], PRES[ISBAD], 'o', ms=10, color='r', mfc='r', alpha=0.7)
        ax1.plot(BBP[innan], PRES[innan], 'o-', ms=3, color='k', mfc='none', alpha=0.7) # <<<<<<<<<<<<<<<<<<

    if QC_TEST_CODE!='E':
        ax1.plot(BBP[innan], PRES[innan], 'o-', ms=3, color='k', mfc='none', alpha=0.7) # <<<<<<<<<<<<<<<<<<
        ax1.plot(BBPmf1[innan], PRES[innan], '-', color='#41F11D', mfc='none', alpha=0.7)
    

    # test-specific additions
    if (QC_TEST_CODE == "A") | (QC_TEST_CODE == "0"):
        ax1.plot(BBP[innan], PRES[innan], 'o-', ms=3, color='k', mfc='none', alpha=0.7) # <<<<<<<<<<<<<<<<<<
        ax1.plot(BBPmf1[innan], PRES[innan], '-', color='#41F11D', mfc='none', alpha=0.7)
        ax1.plot([A_MAX_BBP700, A_MAX_BBP700], [-5, 2000], '--', color='r', mfc='none', alpha=0.7)
        ax1.plot([A_MIN_BBP700, A_MIN_BBP700], [-5, 2000], '--', color='r', mfc='none', alpha=0.7)
        
    if QC_TEST_CODE == "C":
        ax1.plot([C_DEEP_BBP700_THRESH, C_DEEP_BBP700_THRESH], [-5, 2000], '--', color='r', mfc='none', alpha=0.7)

    if QC_TEST_CODE != 'B':
#         ax1.set_xscale('log')
        ax1.set_xlim((-0.001, 0.015))

    elif QC_TEST_CODE =='B':
        ax1.plot([B_RES_THRESHOLD, B_RES_THRESHOLD], [-5, 2000], '--', color='r', mfc='none', alpha=0.7)
        ax1.set_xlim( (1e-6, 1e-2 ) )
        ax1.set_xscale('log')
#         rstd = (np.percentile(rel_res, 84) - np.percentile(rel_res, 16)/2.)
#         ax1.text(-2, 100, str(rstd), fontsize=20)

    if QC_TEST_CODE == "E":
        bin_counts = BBPmf1+10.
        ax1.cla()
        bins = np.linspace(50, 1000, 10)
        ax1.barh(bins-50, bin_counts/2000., height=97, zorder=0 )
        ifail = np.where(bin_counts<10+1)
        ax1.barh(bins[ifail]-50, bin_counts[ifail]/2000., height=97, color='r', zorder=1 )
        ax1.plot(BBP, PRES, 'k.-', zorder=60)   
        
        
#         ax1.plot(BBP, PRES, 'o', ms=10, color='r', mfc='r', alpha=0.7)
    # ax1.set_ylim((np.nanpercentile(O2sat_surf,0.1), np.nanpercentile(O2sat_surf,99.1)))
    # ax1.plot(BBP700[ibad[1],:], Y[ibad[1],:], 'o', ms=2, color='r', mfc='none', alpha=0.7)
    
        
    ax1.grid('on')

    ax1.set_ylim([-5, 2000])    
    ax1.invert_yaxis()

    ax1.set_xlabel('BBP [1/m]', fontsize=20)
    ax1.set_ylabel('PRES [dbars]', fontsize=20)
    ax1.set_title('QC='+QC_TEST_CODE+" "+fn.split('/')[-1], fontsize=20, color='r', fontweight='bold')

    fname = DIR_OUT + "/" + fn.split('/')[-3] + "/" + fn.split('/')[-4] + "_" + fn.split('/')[-1] + "_" + QC_TEST_CODE+ ".png"
    fig.savefig(fname, dpi = 75) 

    # minimise memory leaks
    plt.close(fig)
    gc.collect()
    
# #     plt.ion()  # swtich interactive plotting back on again

    
# #     ipdb.set_trace()

    return 


### Read profile data

#### Open profile data for one WMO

In [45]:
# function to apply tests and plot results (needed in function form for paralle processing)
def QC_wmo(iwmo, VERBOSE=True):   
    
    MAIN_DIR = '/data/datasets/Argo/Original/'
    WORKING_DIR = '/fast_scratch/BGCArgo_plot/source/BBP_RTQC/'
    DIR_OUT = './plots/'
    PLOT = True

    print(iwmo)

    if len(iwmo)==0:
        return
#     if (iwmo.split('/')[0]!='coriolis'):
#         continue
    
    fn2glob = MAIN_DIR + "dac/" + iwmo + "/profiles/" + "B*" + iwmo.split("/")[-1] + "*_[0-9][0-9][0-9].nc"
    fn_single_profiles = np.sort(glob.glob(fn2glob))

    # create dir for output plots
    dout = DIR_OUT + "/" + fn_single_profiles[0].split('/')[-3]
    
    if not os.path.isdir(dout):
        os.mkdir(dout)
        if VERBOSE:
            print("created " + dout)
    else: # remove old plots from this dir
        if VERBOSE:
            print("removing old plots in " + dout + "...")
        oldfn = glob.glob(WORKING_DIR + DIR_OUT + iwmo.split("/")[-1] + "/*png")
        print(WORKING_DIR + DIR_OUT + iwmo.split("/")[-1] + "/*png")
        [os.remove(i) for i in oldfn]   
        if VERBOSE:
            print("...done")
        
    
    all_PRES = []
    all_BBP700 = []
    all_BBP700_QC_flag = []
    all_BBP700_1st_fail = []
    all_BBP700_norm = []

    for fn_p in fn_single_profiles:
        if VERBOSE:
            print(fn_p)
        ds = xr.open_dataset(fn_p)
        
        
        # check if BBP700 is present
        v = set(ds.data_vars)
        if 'BBP700' not in v:
            if VERBOSE:
                print('no BBP700 for this cycle')
            ds.close()
            continue


        # find N_PROF where the BBP700 data are stored     
        tmp_bbp = ds.BBP700.values # note that BBP700[N_PROF,N_LEVELS]
        tmp = [np.any(~np.isnan(tmp_bbp[i][:])) for i in range(tmp_bbp.shape[0])] # find which of the different columns of tmp_bbp has at least one non-NaN element
        if np.any(tmp):
            N_PROF = np.where(tmp)[0][0]
        else:
            if VERBOSE:
                print("this profile has less than 5 data points: skipping ")
            continue
    
    
        BBP700 = ds.BBP700[N_PROF].values
        PRES = ds.PRES[N_PROF].values
        JULD = ds.JULD[N_PROF].values
        
        innan = np.where(~np.isnan(BBP700))[0]

        # compute median filtered profile
        BBP700mf1 = np.zeros(BBP700.shape)*np.nan
#         MEDFILT_WIN = 31
#         BBP700mf1[innan] = medfilt1(BBP700[innan], MEDFILT_WIN)
        BBP700mf1[innan] = adaptive_medfilt1(PRES[innan], BBP700[innan])

        # initialise arrays with QC flags[0,:] = 1 (good data)
        BBP700_QC_flags = np.zeros(BBP700.shape)+1
        BBP700_QC_1st_failed_test = np.full(shape=BBP700.shape, fill_value='0')

        
        # Plot original profile even if no QC flag is raisef
        plot_failed_QC_test(BBP700, BBP700mf1, PRES, BBP700*np.nan, BBP700_QC_flags, BBP700_QC_1st_failed_test, '0', fn_p)
        
        
        # GLOBAL-RANGE TEST for BBP700
        BBP700_QC_flag, BBP700_QC_1st_failed_test = BBP700_Global_range_test(BBP700, BBP700mf1, PRES, BBP700_QC_flags, BBP700_QC_1st_failed_test, fn_p, VERBOSE, PLOT)

        # BBP_NOISY_PROFILE TEST
        BBP700_QC_flag, BBP700_QC_1st_failed_test, rel_res = BBP_Noisy_Profile_test(BBP700, BBP700mf1, PRES, BBP700_QC_flags, BBP700_QC_1st_failed_test, fn_p, VERBOSE, PLOT)

        # HIGH_DEEP_VALUES TEST
        BBP700_QC_flag, BBP700_QC_1st_failed_test = BBP700_High_Deep_Values_test(BBP700mf1, PRES, BBP700_QC_flags, BBP700_QC_1st_failed_test, fn_p, VERBOSE, PLOT)

        # MISSING_DATA TEST
        BBP700_QC_flag, BBP700_QC_1st_failed_test = BBP700_Missing_Data_test(BBP700, PRES, BBP700_QC_flags, BBP700_QC_1st_failed_test, fn_p, VERBOSE, PLOT)
        
        # PROFILE-STEP-CHANGE TEST (e.g., 6902737_262)
        
        # ANIMAL SPIKES

        # GROUNDED_BBP TEST
    
        # STUCK-VALUE TEST

        
        ideep = np.where((np.asarray(PRES)>950.) & (np.asarray(PRES)<1050.))[0]
#         if np.any((iwmo.split('/')[0]=='coriolis') &( np.asarray(BBP700mf1)[ideep]>0.001)):
#             ipdb.set_trace()

        # close dataset
        ds.close()

        # compute BBP700(z) - median(BBP700(z<800))
        BBP700_norm = BBP700 - np.nanmedian(BBP700[PRES>800]) + 1e-3
        all_BBP700_norm.extend(BBP700_norm)
        
        
        # save results in res list
        all_PRES.extend(PRES)
        all_BBP700.extend(BBP700)
        all_BBP700_QC_flag.extend(BBP700_QC_flag)
        all_BBP700_1st_fail.extend(BBP700_QC_1st_failed_test)
#         res.append([PRES, BBP700, BBP700_QC_flag, BBP700_QC_1st_failed_test, JULD])
    
#         ipdb.set_trace()    
    
    # save results in pickled file (https://www.datacamp.com/community/tutorials/pickle-python-tutorial)
    fname = DIR_OUT + fn_p.split('/')[-3] + "/" + fn_p.split('/')[-4] + "_" + fn_p.split('/')[-1].split('.')[0].split('_')[0] +  ".pkl"
    fnout = open(fname,'wb')
    pickle.dump([all_PRES, all_BBP700, all_BBP700_QC_flag, all_BBP700_1st_fail], fnout)
    fnout.close()

    del all_PRES, all_BBP700, all_BBP700_QC_flag, all_BBP700_1st_fail
    gc.collect()
    

In [39]:
WMO_dac[18]

'coriolis/6901862'

In [46]:
QC_wmo(WMO_dac[18])

coriolis/6901862
removing old plots in ./plots//6901862...
/fast_scratch/BGCArgo_plot/source/BBP_RTQC/./plots/6901862/*png
...done
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_001.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_002.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_003.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_004.nc
Failed High_Deep_Values_test
applying QC=3...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_005.nc
Failed High_Deep_Values_test
applying QC=3...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_006.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_007.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_008.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_009.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_010.nc
Failed High_Deep_

Failed High_Deep_Values_test
applying QC=3...
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_094.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_095.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_096.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_097.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_098.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_099.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/

shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_161.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_162.nc
Failed Global_Range_test
applying QC=3...
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_163.nc
Failed Global_Range_test
applying QC=3...
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_164.nc
Failed Global_Range_test
applying QC=3...
Failed BBP_Noisy_Profile_test
applying QC=3...
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_165.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD69

/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_231.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_232.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_233.nc
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_234.nc
Failed High_Deep_Values_test
applying QC=3...
shallow profile: QC=2
Failed Missing_Data_test
applying QC=2...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_235.nc
Failed High_Deep_Values_test
applying QC=3...
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_236.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_237.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_238.nc
Failed Global_Range_test
applying QC=3...
/d

/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_312.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_313.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_314.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_315.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_316.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_317.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_318.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_319.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_320.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_321.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_322.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_323.nc
/data/datasets/Argo/Original/dac/coriolis/6901862/profiles/BD6901862_324.nc
/data/datase

In [17]:
# for iwmo in WMO_dac[0:1]:
#     QC_wmo(iwmo)
from joblib import delayed, Parallel
r = Parallel(n_jobs=7, verbose=40)(delayed(QC_wmo)(iwmo) for iwmo in WMO_dac[:])

# r = Parallel(n_jobs=8, verbose=10)(delayed(np.sqrt)(iwmo) for iwmo in range(10000))



[Parallel(n_jobs=7)]: Using backend LokyBackend with 7 concurrent workers.
[Parallel(n_jobs=7)]: Done   1 tasks      | elapsed:    4.9s
[Parallel(n_jobs=7)]: Done   2 tasks      | elapsed:    6.0s
[Parallel(n_jobs=7)]: Done   3 tasks      | elapsed:    7.1s
[Parallel(n_jobs=7)]: Done   4 tasks      | elapsed:   16.6s
[Parallel(n_jobs=7)]: Done   5 tasks      | elapsed:   21.7s
[Parallel(n_jobs=7)]: Done   6 tasks      | elapsed:   22.4s
[Parallel(n_jobs=7)]: Done   7 tasks      | elapsed:   24.7s
[Parallel(n_jobs=7)]: Done   8 tasks      | elapsed:   27.1s
[Parallel(n_jobs=7)]: Done   9 tasks      | elapsed:   29.6s
[Parallel(n_jobs=7)]: Done  10 tasks      | elapsed:   31.4s
[Parallel(n_jobs=7)]: Done  11 tasks      | elapsed:   36.4s
[Parallel(n_jobs=7)]: Done  12 tasks      | elapsed:   40.0s
[Parallel(n_jobs=7)]: Done  13 tasks      | elapsed:  1.0min
[Parallel(n_jobs=7)]: Done  14 tasks      | elapsed:  1.0min
[Parallel(n_jobs=7)]: Done  15 tasks      | elapsed:  1.1min
[Parallel(

In [18]:
WMO_dac[0:4]
fn_pkl = glob.glob("plots/*/aoml*59035*pkl")
fn_pkl

[]

In [19]:
# find all pickled results
fn_pkl = glob.glob("plots/*/*pkl")

all_PRES = []
all_BBP700 = []
all_BBP700_QC_flag = []
all_BBP700_1st_fail = []

for fn in fn_pkl:
    print(fn)
    infile = open(fn,'rb')
    [tmp_PRES, tmp_BBP700, tmp_BBP700_QC_flag, tmp_BBP700_1st_fail] = pickle.load(infile)
    infile.close()
    
    innan = np.where(~np.isnan(tmp_PRES))[0]
    all_PRES.extend(np.asarray(tmp_PRES)[innan])
    all_BBP700.extend(np.asarray(tmp_BBP700)[innan])
    all_BBP700_QC_flag.extend(np.asarray(tmp_BBP700_QC_flag)[innan])
    all_BBP700_1st_fail.extend(np.asarray(tmp_BBP700_1st_fail)[innan])


plots/5905376/aoml_BD5905376.pkl
plots/6903093/coriolis_BR6903093.pkl
plots/2902114/incois_BD2902114.pkl
plots/6901183/bodc_BR6901183.pkl
plots/5906226/aoml_BR5906226.pkl
plots/6901580/coriolis_BD6901580.pkl
plots/2902195/incois_BR2902195.pkl
plots/5904478/aoml_BD5904478.pkl
plots/3901498/bodc_BR3901498.pkl
plots/2903394/jma_BR2903394.pkl
plots/6901032/coriolis_BD6901032.pkl
plots/2903213/jma_BR2903213.pkl
plots/6902901/coriolis_BD6902901.pkl
plots/5906246/aoml_BR5906246.pkl
plots/6903273/coriolis_BR6903273.pkl
plots/5904181/aoml_BD5904181.pkl
plots/2903669/jma_BR2903669.pkl
plots/2903329/jma_BR2903329.pkl
plots/6901482/coriolis_BD6901482.pkl
plots/1902332/jma_BR1902332.pkl
plots/6901151/bodc_BR6901151.pkl
plots/6901527/coriolis_BD6901527.pkl
plots/2903393/jma_BR2903393.pkl
plots/6902954/coriolis_BR6902954.pkl
plots/5904468/aoml_BD5904468.pkl
plots/2902245/incois_BR2902245.pkl
plots/5905397/csiro_BR5905397.pkl
plots/5905108/aoml_BR5905108.pkl
plots/5905991/aoml_BR5905991.pkl
plots/5905

In [20]:
iQCge3 = np.where(np.asarray(all_BBP700_QC_flag)>=3)[0]
iQCle2 = np.where(np.asarray(all_BBP700_QC_flag)<=2)[0]
# find indices of flagged points
iA = np.where(np.asarray(all_BBP700_1st_fail)[iQCge3]=='A')[0] # global range
iB = np.where(np.asarray(all_BBP700_1st_fail)[iQCge3]=='B')[0] # noisy
iC = np.where(np.asarray(all_BBP700_1st_fail)[iQCge3]=='C')[0] # high deep values
# iD = np.where(np.asarray(all_BBP700_1st_fail)=='D')[0] # 
iE = np.where(np.asarray(all_BBP700_1st_fail)[iQCge3]=='E')[0] # missing values

iall = np.where( (np.asarray(all_BBP700_1st_fail)!='A') & 
                 (np.asarray(all_BBP700_1st_fail)!='B')  &
                 (np.asarray(all_BBP700_1st_fail)!='C') &
#                  (np.asarray(all_BBP700_1st_fail)!='D') &
                 (np.asarray(all_BBP700_1st_fail)!='E'))[0]

itot = np.asarray(all_BBP700).size
print(len(iA)/itot, len(iB)/itot, len(iC)/itot, len(iE)/itot, (itot - len(iQCge3))/itot, len(iall)/itot)



0.024139346218407177 0.003740350815865702 0.03532359345263825 0.011809095559778725 0.9249876139533102 0.8660009572716681


In [21]:
print(len(iA)/itot, len(iB)/itot, len(iC)/itot, len(iE)/itot, (itot - len(iQCge3))/itot, len(iQCle2)/itot)
print([len(all_BBP700_QC_flag), len(all_BBP700_1st_fail)])

0.024139346218407177 0.003740350815865702 0.03532359345263825 0.011809095559778725 0.9249876139533102 0.9249876139533102
[5950244, 5950244]


In [22]:
isurf = np.where((np.array(all_PRES)[iQCle2][::iN]<5) & (np.array(all_BBP700)[iQCle2][::iN]<0))


NameError: name 'iN' is not defined

In [ ]:
# %matplotlib widget
iN = 1#int(iall.size/3.) # step size to make plotting faster
fig = plt.figure(figsize=(20,10))
xmin = 0.0001
xmax = 0.1
xoff = 0.0003
ax1 = fig.add_subplot(1,2,1)
ax1.plot(np.array(all_BBP700)[:][::iN] + xoff, np.array(all_PRES)[:][::iN], 'ro', ms=3, mfc='r', alpha=0.3)
ax1.plot(np.array(all_BBP700)[iQCle2][::iN] + xoff, np.array(all_PRES)[iQCle2][::iN], 'ko', ms=3, alpha=0.03)
ax1.plot((xoff, xoff), (-50, 2000), 'k--')
ax1.set_ylim([-50, 2000])
# ax1.set_xlim([-0.0003, 0.1])
ax1.set_xlim([xmin, xmax])
ax1.invert_yaxis()
ax1.set_xscale('log')
ax1.set_xlabel('BBP700 [m'+'$^{-1}$'+']', fontweight='bold')
ax1.set_ylabel('PRES [dbars]', fontweight='bold')
xts = list(np.asarray([0, 1e-3, 1e-2, 1e-1])+xoff)
xtklbls = ('0',  '10'+'$^{-3}$', '10'+'$^{-2}$', '10'+'$^{-1}$')
plt.xticks(xts, xtklbls)
plt.rcParams.update({'font.size': 20})


gc.collect()
    

In [ ]:
def plot_iQC(x1, x2, y1, y2):


    # %matplotlib widget
    fig = plt.figure(figsize=(20,10))
    xmin = 0.0001
    xmax = 0.1
    xoff = 0.0003
    ax1 = fig.add_subplot(1,2,1)

    ax1.plot(x1 + xoff, y1, 'o', ms=3, alpha=0.03, color='b')
    ax1.plot(x2 + xoff, y2, 'ro', ms=3, alpha=0.1)

    ax1.plot((xoff, xoff), (-50, 2000), 'k--')

    ax1.set_ylim([-50, 2000])
    # ax1.set_xlim([-0.0003, 0.1])
    ax1.set_xlim([xmin, xmax])
    ax1.invert_yaxis()
    ax1.set_xscale('log')
    ax1.set_xlabel('BBP700 [m'+'$^{-1}$'+']', fontweight='bold')
    ax1.set_ylabel('PRES [dbars]', fontweight='bold')
    xts = list(np.asarray([0, 1e-3, 1e-2, 1e-1]) + xoff)
    xtklbls = ('0',  '10'+'$^{-3}$', '10'+'$^{-2}$', '10'+'$^{-1}$')
    plt.xticks(xts, xtklbls)
    plt.rcParams.update({'font.size': 20})

    gc.collect()


In [ ]:
# plots results for Global range test (A)
iN = 1
x1 = np.array(all_BBP700)[:][::iN]
x2 = np.array(all_BBP700)[iQCge3][iA][::]
y1 = np.array(all_PRES)[:][::iN]
y2 = np.array(all_PRES)[iQCge3][iA][::]
plot_iQC(x1, x2, y1, y2)

In [ ]:
# plots results for test (B)
# x1 = np.array(all_BBP700)[:][::iN]
x2 = np.array(all_BBP700)[iQCge3][iB][::]
# y1 = np.array(all_PRES)[:][::iN]
y2 = np.array(all_PRES)[iQCge3][iB][::]
plot_iQC(x1, x2, y1, y2)

In [ ]:
# plots results for test (C)
# x1 = np.array(all_BBP700)[:][::iN]
x2 = np.array(all_BBP700)[iQCge3][iC][::]
# y1 = np.array(all_PRES)[:][::iN]
y2 = np.array(all_PRES)[iQCge3][iC][::]
plot_iQC(x1, x2, y1, y2)

In [ ]:
# plots results for test (E)
# x1 = np.array(all_BBP700)[:][::iN]
x2 = np.array(all_BBP700)[iQCge3][iE][::]
# y1 = np.array(all_PRES)[:][::iN]
y2 = np.array(all_PRES)[iQCge3][iE][::]
plot_iQC(x1, x2, y1, y2)

In [ ]:
# del all_PRES, all_BBP700, all_BBP700_QC_flag, all_BBP700_1st_fail

In [ ]:
# %matplotlib widget
iN = 1#int(iall.size/3.) # step size to make plotting faster
fig = plt.figure(figsize=(20,10))
xmin = 0.0002
xmax = 0.01
xoff = 0.0003
ax1 = fig.add_subplot(1,2,1)
ax1.plot(np.array(all_BBP700)[iQCle2][::iN]+xoff, np.array(all_PRES)[iQCle2][::iN], 'ko', ms=3, alpha=0.02)
ax1.plot((xoff, xoff), (-50, 2000), 'k--')
ax1.set_ylim([-50, 2000])

ax1.set_xlim([xmin, xmax])
ax1.invert_yaxis()
ax1.set_xscale('log')
ax1.set_xlabel('BBP700 [m'+'$^{-1}$'+']', fontweight='bold')
ax1.set_ylabel('PRES [dbars]', fontweight='bold')
xts = list(np.asarray([0, 1e-4, 1e-3, 1e-2]) + xoff)
xtklbls = ('0',  '10'+'$^{-4}$', '10'+'$^{-3}$', '10'+'$^{-2}$')
plt.xticks(xts, xtklbls)
plt.rcParams.update({'font.size': 20})


gc.collect()
    